## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-28-05-31-50 +0000,bbc,Starmer to replace third top aide in less than...,https://www.bbc.com/news/articles/czr6zmd5d0ro...
1,2025-08-28-05-27-49 +0000,nyt,Russian Missile and Drone Attack Kills at Leas...,https://www.nytimes.com/2025/08/28/world/europ...
2,2025-08-28-05-18-28 +0000,nyt,"North Korea’s Leader, Kim Jong-un, to Attend M...",https://www.nytimes.com/2025/08/28/world/asia/...
3,2025-08-28-05-16-13 +0000,bbc,Summer holiday childcare costs a 'struggle',https://www.bbc.com/news/articles/c626x2y2relo...
4,2025-08-28-05-11-34 +0000,bbc,"Eight killed in major Kyiv attack, Zelensky says",https://www.bbc.com/news/articles/cvg3y7m2gz0o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2445/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
95,trump,58
33,school,16
57,new,15
291,shooting,14
91,minneapolis,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
58,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...,155
212,2025-08-27-16-15-57 +0000,nypost,Cracker Barrel changed logo after call with Wh...,https://nypost.com/2025/08/27/business/cracker...,112
25,2025-08-28-03-32-00 +0000,wsj,President Trump on Wednesday fired Robert Prim...,https://www.wsj.com/politics/policy/trump-fire...,110
248,2025-08-27-14-30-00 +0000,wsj,A Beijing-linked espionage campaign that hit U...,https://www.wsj.com/politics/national-security...,104
254,2025-08-27-13-51-00 +0000,nypost,Trump administration announces it’s going to r...,https://nypost.com/2025/08/27/us-news/trump-ad...,103


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
58,155,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...
203,73,2025-08-27-16-45-28 +0000,nyt,The Catholic school where the shooting took pl...,https://www.nytimes.com/live/2025/08/27/us/min...
26,55,2025-08-28-03-19-08 +0000,nypost,Russian ballistic missiles hit Kyiv in large-s...,https://nypost.com/2025/08/27/world-news/russi...
132,49,2025-08-27-21-29-33 +0000,nypost,Zohran Mamdani would easily win NYC mayor race...,https://nypost.com/2025/08/27/us-news/zohran-m...
212,45,2025-08-27-16-15-57 +0000,nypost,Cracker Barrel changed logo after call with Wh...,https://nypost.com/2025/08/27/business/cracker...
185,44,2025-08-27-17-56-38 +0000,latimes,"Cal Fire captain fatally shot his girlfriend, ...",https://www.latimes.com/california/story/2025-...
43,42,2025-08-28-02-12-00 +0000,wsj,Denmark Summons U.S. Envoy Over Suspected Amer...,https://www.wsj.com/world/suspected-american-i...
119,42,2025-08-27-22-03-43 +0000,cbc,CDC director Susan Monarez is being ousted les...,https://www.cbc.ca/news/world/cdc-director-sus...
254,36,2025-08-27-13-51-00 +0000,nypost,Trump administration announces it’s going to r...,https://nypost.com/2025/08/27/us-news/trump-ad...
42,35,2025-08-28-02-12-36 +0000,startribune,"Manifesto, videos from Minneapolis suspect pra...",https://www.startribune.com/manifesto-videos-f...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
